# SWA Differences - SWI + SWISIN Experiments

In [1]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/kdephys/kdephys/plot/acr_plots.mplstyle')

# ---------------------------- EXTRAS --------------------------------#
from acr.utils import swi_subs_exps, sub_probe_locations, sub_exp_types

/home/kdriessen/miniconda3/envs/acrnew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# IMPORTANT PARAMETERS
# --------------------
REBOUND_LENGTH = '1h'
REL_STATE = 'NREM'

In [8]:
sd_df = pd.read_csv('sd_df.csv')
for col in sd_df.columns:
    if "Unnamed" in col:
        del sd_df[col]

In [3]:
def get_rebound_df(subject, exp, reprocess_existing=False, save=True):
    from acr.utils import swi_subs_exps, sub_probe_locations, sub_exp_types
    reb_df = pd.read_csv('reb_df.csv')
    for col in reb_df.columns:
        if "Unnamed" in col:
            del reb_df[col]
    
    already_added = not reb_df.loc[(reb_df.subject==subject)&(reb_df.exp==exp)].empty
    print(already_added)
    if already_added:
        if reprocess_existing:
            reb_df = reb_df.loc[(reb_df.subject!=subject)|(reb_df.exp!=exp)]
        else:
            return
    
    # load some basic information, and the hypnogram
    h = acr.io.load_hypno_full_exp(subject, exp)
    si = acr.info_pipeline.load_subject_info(subject)
    params = acr.info_pipeline.subject_params(subject)
    stores = params['time_stores']
    sort_ids = [f'{exp}-{store}' for store in stores]
    recordings = acr.info_pipeline.get_exp_recs(subject, exp)
    
    # load some temporal information about the rebound, baseline, sd, etc. 
    stim_start, stim_end = acr.stim.stim_bookends(subject, exp)
    reb_start = h.hts(stim_end-pd.Timedelta('15min'), stim_end+pd.Timedelta('1h')).st('NREM').iloc[0].start_time
    if reb_start < stim_end:
        stim_end_hypno = h.loc[(h.start_time<stim_end)&(h.end_time>stim_end)] # if stim time is in the middle of a nrem bout, then it can be the start of the rebound
        if stim_end_hypno.state.values[0] == 'NREM':
            reb_start = stim_end
        else:
            raise ValueError('Rebound start time is before stim end time, need to inspect')

    assert reb_start >= stim_end, 'Rebound start time is before stim end time'

    bl_start_actual = si["rec_times"][f'{exp}-bl']["start"]
    bl_day = bl_start_actual.split("T")[0]
    bl_start = pd.Timestamp(bl_day + "T09:00:00")

    if f'{exp}-sd' in si['rec_times'].keys():
        sd_rec = f'{exp}-sd'
        sd_end = pd.Timestamp(si['rec_times'][sd_rec]['end'])
    else:
        sd_rec = exp
        sd_end = stim_start
    sd_start_actual = pd.Timestamp(si['rec_times'][sd_rec]['start'])
    sd_day = si['rec_times'][sd_rec]['start'].split("T")[0]
    sd_start = pd.Timestamp(sd_day + "T09:00:00")
    
    # Load the BANDPOWER DATA
    #-------------------------------
    bp = acr.io.load_concat_bandpower(subject, recordings, stores, hypno=True);
    bp_rel = kde.xr.utils.rel_by_store(bp, state=REL_STATE, t1=None, t2=None);

    # Get the rebound values:
    rebound_dur = pd.Timedelta(REBOUND_LENGTH)
    reb_bp = bp_rel.ts(reb_start, reb_start+rebound_dur) 
    bp_df = kde.xr.spectral.bp_melt(reb_bp.to_dataframe().reset_index()) #convert to dataframe
    bp_df['region'] = sub_probe_locations[subject]
    bp_df['exp_type'] = sub_exp_types[subject]
    bp_df['subject'] = subject
    bp_df['exp'] = exp
    reb_df = pd.concat([reb_df, bp_df])
    if save:
        reb_df.to_csv('reb_df.csv') 
    return

In [3]:
def get_sd_df(subject, exp, reprocess_existing=False, save=True):
    from acr.utils import swi_subs_exps, sub_probe_locations, sub_exp_types
    sd_df = pd.read_csv('swi_df.csv')
    for col in sd_df.columns:
        if "Unnamed" in col:
            del sd_df[col]
    if sd_df.empty:
        already_added = False
    else:
        print('wrong')
        already_added = not sd_df.loc[(sd_df.subject==subject)&(sd_df.exp==exp)].empty
    print(already_added)
    if already_added:
        if reprocess_existing:
            sd_df = sd_df.loc[(sd_df.subject!=subject)|(sd_df.exp!=exp)]
        else:
            return
    
    # load some basic information, and the hypnogram
    h = acr.io.load_hypno_full_exp(subject, exp)
    si = acr.info_pipeline.load_subject_info(subject)
    params = acr.info_pipeline.subject_params(subject)
    stores = params['time_stores']
    sort_ids = [f'{exp}-{store}' for store in stores]
    recordings = acr.info_pipeline.get_exp_recs(subject, exp)
    
    # load some temporal information about the rebound, baseline, sd, etc. 
    stim_start, stim_end = acr.stim.stim_bookends(subject, exp)
    reb_start = h.hts(stim_end-pd.Timedelta('15min'), stim_end+pd.Timedelta('1h')).st('NREM').iloc[0].start_time
    if reb_start < stim_end:
        stim_end_hypno = h.loc[(h.start_time<stim_end)&(h.end_time>stim_end)] # if stim time is in the middle of a nrem bout, then it can be the start of the rebound
        if stim_end_hypno.state.values[0] == 'NREM':
            reb_start = stim_end
        else:
            raise ValueError('Rebound start time is before stim end time, need to inspect')

    assert reb_start >= stim_end, 'Rebound start time is before stim end time'

    bl_start_actual = si["rec_times"][f'{exp}-bl']["start"]
    bl_day = bl_start_actual.split("T")[0]
    bl_start = pd.Timestamp(bl_day + "T09:00:00")

    if f'{exp}-sd' in si['rec_times'].keys():
        sd_rec = f'{exp}-sd'
        sd_end = pd.Timestamp(si['rec_times'][sd_rec]['end'])
    else:
        sd_rec = exp
        sd_end = stim_start
    sd_start_actual = pd.Timestamp(si['rec_times'][sd_rec]['start'])
    sd_day = si['rec_times'][sd_rec]['start'].split("T")[0]
    sd_start = pd.Timestamp(sd_day + "T09:00:00")
    
    # Load the BANDPOWER DATA
    #-------------------------------
    bp = acr.io.load_concat_bandpower(subject, recordings, stores, hypno=True);
    bp_rel = kde.xr.utils.rel_by_store(bp, state=REL_STATE, t1=None, t2=None);

    # Get the SD values:
    sd_bp = bp_rel.ts(stim_start, stim_end) 
    bp_df = kde.xr.spectral.bp_melt(sd_bp.to_dataframe().reset_index()) #convert to dataframe
    bp_df['region'] = sub_probe_locations[subject]
    bp_df['exp_type'] = sub_exp_types[subject]
    bp_df['subject'] = subject
    bp_df['exp'] = exp
    sd_df = pd.concat([sd_df, bp_df])
    if save:
        sd_df.to_csv('swi_df.csv') 
    return

In [4]:
for subject in swi_subs_exps:
    for exp in swi_subs_exps[subject]:
        get_sd_df(subject, exp)

False
swi-bl was missing recording and store coordinates, added them
swi-sd was missing recording and store coordinates, added them
swi was missing recording and store coordinates, added them
swi-bl was missing recording and store coordinates, added them
swi-sd was missing recording and store coordinates, added them
swi was missing recording and store coordinates, added them
wrong
False
swi-bl was missing recording and store coordinates, added them
swi-sd was missing recording and store coordinates, added them
swi was missing recording and store coordinates, added them
swi-post was missing recording and store coordinates, added them
swi-bl was missing recording and store coordinates, added them
swi-sd was missing recording and store coordinates, added them
swi was missing recording and store coordinates, added them
swi-post was missing recording and store coordinates, added them
wrong
False
swi2-bl was missing recording and store coordinates, added them
swi2-sd was missing recording an